# Profitable App Profiles for the App Store and Google Play Markets

Our goal here is figuring out which types of apps would be the most profitable for a hypothetical company that makes free Android and IOS apps and relies on in-app advertisements for revenue. The revenue model here is dictated by the amount of users that actively use the app, as more people using the app would result in higher engagement with the advertisements.

So here we will go through the different types of apps offered on the Google Play Store and Apple App Store in order to figure out which apps would attract the most users.

There are over 2 million Android apps on the Google Play Store and over 2 million IOS apps on the App Store. Collecting data for these millions of apps will require a significant investment, so instead we will be using a sample of the data instead.

We have 2 different data sets, one for Android applications and another for IOS applications.

We can see the documentation for our Android app data set containing ten thousand apps [here](https://www.kaggle.com/lava18/google-play-store-apps), and the documentation for our IOS app data set containing seven thousand apps [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps).



In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print("\n")
    if rows_and_columns:
        print("Number of rows:", len(dataset))
        print("Number of columns:", len(dataset[0]))

In [3]:
#Google Play Store
opened_file=open("googleplaystore.csv")
from csv import reader
read_file=reader(opened_file)
android=list(read_file)
android_header=android[0]
android_apps=android[1:]

explore_data(android_apps,0,5,True)

#Apple App Store
opened_file=open("AppleStore.csv")
read_file=reader(opened_file)
ios=list(read_file)
ios_header=ios[0]
ios_apps=ios[1:]

explore_data(ios_apps,0,5,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 10841
Number of columns: 13
['284882215', 'Facebook', '38987980

The names for the columns of the Play Store data set are self explanatory, however the names for the App Store columns is less evident.

The documentation for the App Store data set can be seen [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps). Below is a table showing what each column represents.


| Column Name      | Description |
| ----------- | ----------- |
| id      | App ID       |
| track_name   | App Name        |
| size_bytes      | Size in bytes       |
| currency   | Currency type        |
| price      | Price amount       |
| rating_count_tot   | User Rating counts (for all versions)        |
| rating_count_ver      | User Rating counts (for current version)       |
| user_rating   | Average User Rating value (for all versions)        |
| user_rating_ver      | Average User Rating value (for current version)         |
| ver   | Latest version code        |
| cont_rating      | Content Rating       |
| prime_genre   | Primary Genre        |
| sup_devices.num      | Number of supporting devices       |
| ipadSc_urls.num   | Number of screenshots showed for display        |
| lang.num      | Number of supported languages       |
| vpp_lic   | Vpp Device Based Licensing Enabled        |


In [7]:
# Here we can see that the Android app with an index of 10472 has a missing "Genre" rating.
# This could create an issue going forward, so we shall remove the offending row.

print(android[10473])  # incorrect row
print('\n')
print(android_header)  # header
print('\n')
print(android[0])      # correct row

print(len(android))
del android[10473]  # don't run this more than once
print(len(android))

android_header=android[0]
android_apps=android[1:]


# for app in android_apps:
#     app_index=-1
#     app_index+=1
#     if len(app) != len(android_header):
#         print(app)
#         del android_apps[app_index]
# for app in ios_apps:
#     if len(app) != len(ios_header):
#         print(app)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
10842
10841


In [8]:
print(len(android_apps))


10840


In [9]:
#We can check and see if we have any duplicate rows in our data sets.

duplicate_apps=[]
unique_apps=[]

for app in ios_apps:
    name=app[0]
    
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print(len(duplicate_apps))

duplicate_apps=[]
unique_apps=[]

for app in android_apps:
    name=app[0]
    
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print(len(duplicate_apps))

0
1181


# Duplicate Apps

We can see that we have over one thousand duplicate Android Apps, while we have no duplicate IOS apps. We will remove the duplicate applications based on the number of reviews, deleting the versions of the apps that have the least number of reviews.

In [20]:
expected_length=len(android_apps)-len(duplicate_apps)

reviews_max={}

for app in android_apps:
    name=app[0]
    n_reviews=float(app[3])
    if name in reviews_max and reviews_max[name]<n_reviews:
        reviews_max[name]=n_reviews
    elif name not in reviews_max:
        reviews_max[name]=n_reviews
        
print(expected_length)
print(len(reviews_max))

android_clean=[]
already_added=[]

for app in android_apps:
    name=app[0]
    n_reviews=float(app[3])
    
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

print(len(android_clean))

9659
9659
9659


# Removing Duplicate Apps

Above, we removed all duplicate apps found in our original android_apps data set. We now have a new data set known as android_clean, which has been purged of all duplicate applications.

In [21]:
explore_data(android_clean,0,7,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+

# Removing duplicate data

The company will be making English apps, thus apps that are not in English will be less relevant to our findings. We shall remove any applications with names that are not English.

In [27]:
def is_english(input_string):
    for i in input_string:
        if ord(i)>127:
            return False
    return True
        
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
False
False


In [35]:
print(ord("😜"))
print(ord("™"))

128540
8482


As we saw above, some English apps might contain characters outside of the above range, such as apps that contain Emojis in the name.

Thus we will instead use a function that only returns False if there are at least 3 characters outside of the range.

In [64]:
def is_english(input_string):
    non_eng=0
    for i in input_string:
        if ord(i)>127:
            non_eng+=1
        if non_eng>3:
            return False
    return True
        
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
True
True


In [80]:
android_english=[]
ios_english=[]

def eng_dataset(dataset,new_dataset,index=0):
    for app in dataset:
        name=app[index]
        if is_english(name):
            new_dataset.append(app)
            
eng_dataset(dataset=android_clean,new_dataset=android_english,index=0)

eng_dataset(ios_apps,ios_english,index=1)

explore_data(android_english,0,5,True)
explore_data(ios_english,0,5,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9614
Number of columns: 13
['284882215', 'Facebook', '389879808', 'USD', '0

# Free Apps

As mentioned previously, we are only interested in Free apps. Thus, now we can work to remove all the paid apps 
from our data set.

In [114]:
android_final=[]
ios_final=[]

def is_free(dataset,new_dataset,index):
    for app in dataset:
        price=app[index]
        # 7 for Android, 4 for IOS
        if price == "0.0" or price == "0":
            new_dataset.append(app)
            
is_free(android_english,android_final,7)
is_free(ios_english,ios_final,4)



In [116]:
explore_data(android_final,0,10,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+

In [117]:
explore_data(ios_final,0,10,True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


['429047995', 'Pinterest', '74778624', 'USD', '0.0', '1061624', '1814', '4.5', '4.0', '6.26', '12+', 'Social Networking', '37', '5', '27', '1']


['282935706', 'Bible', '92774400', 'USD', '0.0', '985920', '5320', '4.5', '5.0', '7.5.1', '4+', 'Reference', '37', '5', '45', '1']


['5538347

# Aim

Now that we have cleaned our data, we can focus more on our original aim.

The idea for the app would be first creating a minimal Android version, then developing it further given a good user response, then build an IOS version in 6 months given that it is profitable.

So we will find genres of apps that should be successful both on Android and on IOS. We first need to see the different Genres and see which would be the most successful. We will do this below by generating a Frequency table.

In [119]:
def display_table(dataset,index):
    table=freq_table(dataset,index)
    table_display=[]
    
    for key in table:
        key_val_as_tuple = (table[key],key)
        table_display.append(key_val_as_tuple)
    table_sorted=sorted(table_display,reverse=True)
    
    for entry in table_sorted:
        print(entry[1],":",entry[0])

In [134]:
def freq_table(dataset,index):
    freq={}
    tot=0
    for app in dataset:
        tot+=1
        val=app[index]
        if val in freq:
            freq[val]+=1
        else:
            freq[val]=1

    table_percents={}
    for key in freq:
        table_percents[key]=(freq[key]/tot)*100
    return table_percents

In [137]:
display_table(android_final,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [142]:
display_table(android_final,9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In [144]:
display_table(ios,11)

Games : 53.65379272020006
Entertainment : 7.4326201722700755
Education : 6.293414837454849
Photo & Video : 4.848569046957488
Utilities : 3.445401500416782
Health & Fitness : 2.5006946373992776
Productivity : 2.4729091414281745
Social Networking : 2.3200889135871074
Lifestyle : 2.0005557099194218
Music : 1.9171992220061127
Shopping : 1.694915254237288
Sports : 1.5837732703528755
Book : 1.5559877743817727
Finance : 1.4448457904973604
Travel : 1.125312586829675
News : 1.0419560989163656
Weather : 1.0002778549597109
Reference : 0.8891358710752986
Food & Drink : 0.875243123089747
Business : 0.7918866351764378
Navigation : 0.6390664073353709
Medical : 0.31953320366768545
Catalogs : 0.13892747985551543
prime_genre : 0.01389274798555154


# Genres

Above, we can see the results of our frequncy tables. We must first keep in mind that this is not representative of the App Store and Google Play stores as a whole. We are only looking at Free English apps from our sample.

First let's focus on the Android data, given that the app would first be released on the Google Play Store. In our data set, we have 2 relevant columns, "Category" and "Genre". First, looking at the "Category" column, we can see that games make up a large portion of our applications. The "Family" category is actually populated by a large amount of games, as we can see when we search up the aforementioned category. That combined with the "Games" category makes up nearly a third of our total applications. Next we can look at the "Genre" column. The distinction seems unclear, however the Genre column contains a much larger variety, as well as more details. Since we are looking for a general overview, we should stick to the Category column.

Next, we can look at the IOS data. This seems to be much more clear. The "Games" genre makes up a majority of the applications on the App Store. The IOS App Store also contains much more entertainment oriented applications than the Google Play Store.

However, this is not enough information to decide what kind of app we should make. It is possible that Games might be the most popular category in terms of volume, but they might not be the best in terms of ratings. It's possible that there are simply more Games because they could be easier to make, or because developers might make more free games because they are more interested in Game Development than they are in developing an app for, for example, "Medicine". This also doesn't show us which genres have the most users.

Thus, we still do not have enough information to suggest an app profile.

In [140]:
print(android_header)
print("\n")
print(ios_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [186]:
prime_gen=freq_table(ios_final,11)

def user_rtg(dataset,genre_index,rating_index):
    prime_gen=freq_table(dataset,genre_index)
    for genre in prime_gen:
        total=0
        len_genre=0
        
        for app in dataset:
            genre_app=app[genre_index]
            
            if genre==genre_app:
                total+=float(app[rating_index])
                len_genre+=1
        avg_ratings=total/len_genre
        print(genre + ": " + str(avg_ratings),  len_genre)

In [189]:
user_rtg(ios_final,-5,5)

print("\n")


user_rtg(android_final,1,3)


ART_AND_DESIGN: 24699.42105263158 57
AUTO_AND_VEHICLES: 14140.280487804877 82
BEAUTY: 7476.226415094339 53
BOOKS_AND_REFERENCE: 87995.06842105264 190
BUSINESS: 24239.727272727272 407
COMICS: 42585.61818181818 55
COMMUNICATION: 995608.4634146341 287
DATING: 21953.272727272728 165
EDUCATION: 56293.09708737864 103
ENTERTAINMENT: 301752.24705882353 85
EVENTS: 2555.84126984127 63
FINANCE: 38535.8993902439 328
FOOD_AND_DRINK: 57478.79090909091 110
HEALTH_AND_FITNESS: 78094.9706959707 273
HOUSE_AND_HOME: 26435.465753424658 73
LIBRARIES_AND_DEMO: 10925.807228915663 83
LIFESTYLE: 33921.82369942196 346
GAME: 683523.8445475638 862
FAMILY: 113142.99821002387 1676
MEDICAL: 3730.1533546325877 313
SOCIAL: 965830.9872881356 236
SHOPPING: 223887.34673366835 199
PHOTOGRAPHY: 404081.3754789272 261
SPORTS: 116938.6146179402 301
TRAVEL_AND_LOCAL: 129484.42512077295 207
TOOLS: 305732.8973333333 750
PERSONALIZATION: 181122.31632653062 294
PRODUCTIVITY: 160634.5420289855 345
PARENTING: 16378.706896551725 58
W

# App Store User Ratings
 
With the IOS App Store dataset, we do not have access to the number of installs, so we will look at the next best thing which is the number of reviews. 

We can see that the category with the most number of reviews/app is Navigation. However, this is largely dominated by a few apps such as Google Maps and Waze. There are only 6 apps in this category and 2 of them are the aforementioned giants, both owned by Google. It might not be wise to attempt to make an app that strives to rival Google Maps and Waze, but perhaps something more specialized would be a good idea. Perhaps the map of a particular city that can go into more details than Google Maps or Waze does.

Social Networking apps fare a similar faith. While they have many reviews per app and there are 106 apps in this category, the category is still dominated by a few dozen giants. Regardless, Social Media apps would not be a good choice as they would need to hit an inflection point before they take off in popularity which would be difficult to do, as people won't use these apps unless they know people using them.

Reference apps could be a good idea. They are next in line in terms of reviews per app and while they have a few giants, they aren't as heavily dominated by them as Social Networking and Navigation apps were. Perhaps it could be a religous app or a guide of some sort.

In [200]:
def print_genre(dataset,index,col_name):
    for app in dataset:
        if app[index]==col_name:
            print(app)
            print("\n")
print_genre(ios_final,-5,"Navigation")
print_genre(ios_final,-5,"Social Networking")
print_genre(ios_final,-5,"Reference")

['323229106', 'Waze - GPS Navigation, Maps & Real-time Traffic', '94139392', 'USD', '0.0', '345046', '3040', '4.5', '4.5', '4.24', '4+', 'Navigation', '37', '5', '36', '1']


['585027354', 'Google Maps - Navigation & Transit', '120232960', 'USD', '0.0', '154911', '1253', '4.5', '4.0', '4.31.1', '12+', 'Navigation', '37', '5', '34', '1']


['329541503', 'Geocaching®', '108166144', 'USD', '0.0', '12811', '134', '3.5', '1.5', '5.3', '4+', 'Navigation', '37', '0', '22', '1']


['504677517', 'CoPilot GPS – Car Navigation & Offline Maps', '82534400', 'USD', '0.0', '3582', '70', '4.0', '3.5', '10.0.0.984', '4+', 'Navigation', '38', '5', '25', '1']


['344176018', 'ImmobilienScout24: Real Estate Search in Germany', '126867456', 'USD', '0.0', '187', '0', '3.5', '0.0', '9.5', '4+', 'Navigation', '37', '5', '3', '1']


['463431091', 'Railway Route Search', '46950400', 'USD', '0.0', '5', '0', '3.0', '0.0', '3.17.1', '4+', 'Navigation', '37', '0', '1', '1']


['284882215', 'Facebook', '389879808', 

In [196]:
user_rtg(android_final,1,3)


ART_AND_DESIGN: 24699.42105263158 57
AUTO_AND_VEHICLES: 14140.280487804877 82
BEAUTY: 7476.226415094339 53
BOOKS_AND_REFERENCE: 87995.06842105264 190
BUSINESS: 24239.727272727272 407
COMICS: 42585.61818181818 55
COMMUNICATION: 995608.4634146341 287
DATING: 21953.272727272728 165
EDUCATION: 56293.09708737864 103
ENTERTAINMENT: 301752.24705882353 85
EVENTS: 2555.84126984127 63
FINANCE: 38535.8993902439 328
FOOD_AND_DRINK: 57478.79090909091 110
HEALTH_AND_FITNESS: 78094.9706959707 273
HOUSE_AND_HOME: 26435.465753424658 73
LIBRARIES_AND_DEMO: 10925.807228915663 83
LIFESTYLE: 33921.82369942196 346
GAME: 683523.8445475638 862
FAMILY: 113142.99821002387 1676
MEDICAL: 3730.1533546325877 313
SOCIAL: 965830.9872881356 236
SHOPPING: 223887.34673366835 199
PHOTOGRAPHY: 404081.3754789272 261
SPORTS: 116938.6146179402 301
TRAVEL_AND_LOCAL: 129484.42512077295 207
TOOLS: 305732.8973333333 750
PERSONALIZATION: 181122.31632653062 294
PRODUCTIVITY: 160634.5420289855 345
PARENTING: 16378.706896551725 58
W

# Play Store User Ratings

While we have access to Play Store install numbers, they are not as exact as the user ratings column. Thus we will look at both. Also, I believe that it could be argued that the number of reviews is even more important than the number of installs for our case as having more reviews could imply that the user is spending more time on the app.

Similar to the IOS data, we see Navigation and Social Media at the top. While there are more apps in each of these categories here compared to in IOS, they are still dominated by a few apps. However, perhaps there also exists a niche for a more specified map app as mentioned for IOS. Travel apps also tend to have a high amount of ratings, so perhaps a niche app that covers these two markets would do well.


In [205]:
print_genre(android_final,1,"MAPS_AND_NAVIGATION")
print_genre(android_final,1,"SOCIAL")
print_genre(android_final,1,"TRAVEL_AND_LOCAL")

['Waze - GPS, Maps, Traffic Alerts & Live Navigation', 'MAPS_AND_NAVIGATION', '4.6', '7232629', 'Varies with device', '100,000,000+', 'Free', '0', 'Everyone', 'Maps & Navigation', 'July 29, 2018', 'Varies with device', 'Varies with device']


['T map (te map, T map, navigation)', 'MAPS_AND_NAVIGATION', '4.2', '15681', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Maps & Navigation', 'August 2, 2018', 'Varies with device', 'Varies with device']


['MapQuest: Directions, Maps, GPS & Navigation', 'MAPS_AND_NAVIGATION', '4.1', '53481', 'Varies with device', '10,000,000+', 'Free', '0', 'Everyone', 'Maps & Navigation', 'May 22, 2018', 'Varies with device', 'Varies with device']


['Yahoo! transit guide free timetable, operation information, transfer search', 'MAPS_AND_NAVIGATION', '4.4', '104800', '22M', '10,000,000+', 'Free', '0', 'Everyone', 'Maps & Navigation', 'July 26, 2018', '7.0.1', '4.0 and up']


['乗換NAVITIME\u3000Timetable & Route Search in Japan Tokyo', 'MAPS_AND_N

In [208]:
freq_table(android_final,1)

{'ART_AND_DESIGN': 0.6430505415162455,
 'AUTO_AND_VEHICLES': 0.9250902527075812,
 'BEAUTY': 0.5979241877256317,
 'BOOKS_AND_REFERENCE': 2.1435018050541514,
 'BUSINESS': 4.591606498194946,
 'COMICS': 0.6204873646209386,
 'COMMUNICATION': 3.2378158844765346,
 'DATING': 1.861462093862816,
 'EDUCATION': 1.1620036101083033,
 'ENTERTAINMENT': 0.9589350180505415,
 'EVENTS': 0.7107400722021661,
 'FINANCE': 3.7003610108303246,
 'FOOD_AND_DRINK': 1.2409747292418771,
 'HEALTH_AND_FITNESS': 3.0798736462093865,
 'HOUSE_AND_HOME': 0.8235559566787004,
 'LIBRARIES_AND_DEMO': 0.9363718411552346,
 'LIFESTYLE': 3.9034296028880866,
 'GAME': 9.724729241877256,
 'FAMILY': 18.907942238267147,
 'MEDICAL': 3.531137184115524,
 'SOCIAL': 2.6624548736462095,
 'SHOPPING': 2.2450361010830324,
 'PHOTOGRAPHY': 2.944494584837545,
 'SPORTS': 3.395758122743682,
 'TRAVEL_AND_LOCAL': 2.33528880866426,
 'TOOLS': 8.461191335740072,
 'PERSONALIZATION': 3.3167870036101084,
 'PRODUCTIVITY': 3.892148014440433,
 'PARENTING': 0.6